In [3]:
pip install flatten_json 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#Do you know the number of levels Y/n
import pandas as pd
import requests
import time
import json
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import datasets
import numpy as np
from flatten_json import flatten
# Import the GaussianMixture class
from sklearn.mixture import GaussianMixture




In [5]:
def question_expertise_level():
    value_of_k = 0
    i = 0
    while i < 2:
        answer = input("Do you Know the Experise Level? (yes or no) ")
        if any(answer.lower() == f for f in ["yes", 'y', '1', 'ye']):
          # If yes we call the KMeans cluster method here
            value_of_k = int(answer)
            break

        elif any(answer.lower() == f for f in ['no', 'n', '0']):
          # if no we call the GMM Cluster Method here
            value_of_k = 2
            print(value_of_k)
            break
        else:
            i += 1
            if i < 2:
                print('Please enter yes or no')
            else:
                print("Nothing done")
    return value_of_k  

In [7]:

# Either Python or Tensorflow ?
#import the dataset for the TENSORFLOW tags from the STACKEXCHANGE API

def query_stack_exchange():
  # Ask for the hashtag numbers
  number_of_hashtags = input("Do you know the number of dimensions of expertise you wish to consider?")
  # Ask for the hashtag Values
  hashtag_values = input("Enter the dimensions of expertise as a comma separated list: Usage tensorflow,python ")
  
  # Handle the number of users to consider clustering
  number_of_pages = input("How many users will be considered for comparison in the clustering: Note minimum is 100 users")
  if(number_of_pages > 100):
    number_of_pages = (number_of_pages//100)
  else:
    #  number of pages being set to one is equivalent to 100 users
    number_of_pages = 1 

   
  # # call the function which will ask the user if they know the expertise level or not
  # value_of_k = question_expertise_level()

  complete_data=[]
  dfs = []
  for i in range (int(number_of_pages)):
    response = requests.get("https://api.stackexchange.com/2.3/search/advanced?order=desc&sort=activity&q=" + str(hashtag_values) + "&site=stackoverflow&filter=!*MZqiH2P51Zpclr2&pagesize=100&page=" + str(i + 1))
    newData =json.loads(response.text)
    # newData = flatten(newData_unflatten)
    # print(newData)
    for item in newData["items"]:
        # item = flatten(item)
        # print(item)
        complete_data.append(item)
        dfs.append(pd.DataFrame([item]))
    print("Processed page " + str(i + 1) + ", returned " + str(response))
    time.sleep(2) # timeout not to be rate-limited
  df = pd.concat(dfs, ignore_index=True, sort=False)
  print(df)
  dataStackexchange = df.to_csv('dataStackExchange.csv', encoding='utf-8', index=False)
  #import the dataset for the 150 Elements
  dataset_path = "dataStackExchange.csv"
  dataset = pd.read_csv(dataset_path)#, error_bad_lines=False)

  # which will ask the user if they know the expertise level or not
  answer = input("Do you have a defined number of expertise levels?  (yes or no) ")
  # If yes we call the KMeans cluster method here
  if any(answer.lower() == f for f in ["yes", 'y', '1', 'ye']): 
    value_of_k = int(answer)
    # Fill missing values with mean column values in the data set
    dataset.fillna(dataset.mean(), inplace=True)
    #select the columns you wanna train your data with SET 2:[Upvotes, Downvotes]
    X = df.loc[:, ['down_vote_count', 'up_vote_count']].values
    # #Transform the data
    # df = pca.fit_transform(X)
    # when it is not-manual it is giving me an error 
    kmeans5 = KMeans(n_clusters = value_of_k)
    y_kmeans5 = kmeans5.fit_predict(X)
    print(y_kmeans5)
    dataset['cluster'] = y_kmeans5
    #Getting unique label
    u_labels = np.unique(y_kmeans5)
    print(u_labels)
    #plotting the results:
 
  # if no we call the GMM Cluster Method here
  elif any(answer.lower() == f for f in ['no', 'n', '0']): 
    print("That's okay!")
    print("We will help you pick the best cluster number using Gaussian Mixture Model(GMM) which is a clustering algorithm")
    # df = pd.read_csv('usersCombined_TF.csv')
    X = df.loc[:, ['down_vote_count', 'up_vote_count']].values
    
    # Set up a range of cluster numbers to try
    n_range = range(2,11)

    # Create empty lists to store the BIC and AIC values
    bic_score = []
    aic_score = []

    # Loop through the range and fit a model
    for n in n_range:
      gm = GaussianMixture(n_components=n, 
                          random_state=123, 
                         n_init=10)
      gm.fit(X)
      # Append the BIC and AIC to the respective lists
      bic_score.append(gm.bic(X))
      aic_score.append(gm.aic(X))
    # Plot the BIC and AIC values together
    fig, ax = plt.subplots(figsize=(12,8),nrows=1)
    ax.plot(n_range, bic_score, '-o', color='orange')
    ax.plot(n_range, aic_score, '-o', color='green')
    ax.set(xlabel='Number of Clusters', ylabel='Score')
    ax.set_xticks(n_range)
    ax.set_title('BIC and AIC Scores Per Number Of Clusters')
    

    print("HINT: CHOOSE THE MOST most appropriate cluster number by using the Cluster Value, after which the chart really levels off.")
    gmm_cluster_value = input("Enter the most appropriate cluster value based on the above chart values")
    X = df.loc[:, ['down_vote_count', 'up_vote_count']].values
    gm = GaussianMixture(n_components= int(gmm_cluster_value), random_state=123, n_init=10)
    preds = gm.fit_predict(X)
 
  # # Fill missing values with mean column values in the data set
  # dataset.fillna(dataset.mean(), inplace=True)
  # #select the columns you wanna train your data with SET 2:[Upvotes, Downvotes]
  # X = df.loc[:, ['down_vote_count', 'up_vote_count']].values
  # # #Transform the data
  # # df = pca.fit_transform(X)
  # # when it is not-manual it is giving me an error 
  # kmeans5 = KMeans(n_clusters=2)
  # y_kmeans5 = kmeans5.fit_predict(X)
  # print(y_kmeans5)
  # dataset['cluster'] = y_kmeans5
  # #Getting unique label
  # u_labels = np.unique(y_kmeans5)
  # print(u_labels)
  # #plotting the results:
 
  # for i in u_labels:
  plt.scatter(dataset.cluster == 0 , dataset.cluster == 1, label = i)
  plt.legend()
  plt.show()
  dataset[dataset.cluster == 0].to_csv("set1_First_Cluster_Tensorflow.csv")
  dataset[dataset.cluster == 1].to_csv("set1_Second_Cluster_Tensorflow.csv")
  return newData

newData_unflattned = query_stack_exchange()
# cluster_method()
#filter rows of original data
# filtered_label0 = [dataset.cluster == 0]
 
# filtered_label1 = [dataset.cluster == 1]
 
# #Plotting the results
# plt.scatter(filtered_label0[:,0] , filtered_label0[:,1] , color = 'red')
# plt.scatter(filtered_label1[:,0] , filtered_label1[:,1] , color = 'black')
# plt.show()






Do you know the number of dimensions of expertise you wish to consider?2
Enter the dimensions of expertise as a comma separated list: Usage tensorflow,python java,css
How many users will be considered for comparison in the clustering: Note minimum is 100 users200


TypeError: ignored

In [10]:
def query_user_id():

  # newData_flattened = flatten(newData_unflattned)
  dfs = []
  for item in newData_unflattned["items"]:
     item = flatten(item)
    #  print(item)
     dfs.append(pd.DataFrame([item]))
  df = pd.concat(dfs, ignore_index=True, sort=False)
  dataStackexchange = df.to_csv('dataStackExchange.csv', encoding='utf-8', index=False)
  #import the dataset for the 150 Elements
  dataset_path = "dataStackExchange.csv"
  dataset = pd.read_csv(dataset_path)#, error_bad_lines=False)
  # Fill missing values with mean column values in the data set
  dataset.fillna(dataset.mean(), inplace=True)
  # convert values to int
  dataset = dataset.astype({"owner_account_id":"int","owner_reputation":"int", "owner_user_id": "int"})
  #select the columns you wanna train your data with SET 2:[Upvotes, Downvotes]
  X = df.loc[:, ['down_vote_count', 'up_vote_count']].values
  # #Transform the data
  # df = pca.fit_transform(X)
  # when it is not-manual it is giving me an error 
  kmeans5 = KMeans(n_clusters=2)
  y_kmeans5 = kmeans5.fit_predict(X)
  print(y_kmeans5)
  dataset['cluster'] = y_kmeans5   
  # dataset = dataset.astype({"owner_account_id":"int","owner_reputation":"int", "owner_user_id": "int"})
  print(dataset)
  user_id = input("Enter the is of the user you wish to query for: ")
  # ddata = dataset.loc[(dataset['owner_user_id'] == user_id) & (dataset['cluster'] == 0)]
  print(dataset.loc[dataset['cluster'] == 1]])
  # if dataset[(dataset.owner_user_id == user_id) & (dataset.cluster == 0)].all():
  #   print("Yes, USER ID found in Cluster 0" )
  # if dataset[(dataset.owner_user_id == user_id) & (dataset.cluster == 1)].all():
  #   print("Yes, USER ID found in Cluster 1" )
  # else:
  #   print("user is not in clustered data")

query_user_id()
# # Kmeans clustering

# # Output a percentage

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  from ipykernel import kernelapp as app


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
        tags_0        tags_1                  tags_2  owner_account_id  \
0       python    tensorflow        object-detection          15315560   
1       python    tensorflow        machine-learning          11396232   
2       python    python-3.x              tensorflow          23165717   
3       python    tensorflow                     NaN          14742883   
4       python    tensorflow        machine-learning          22181347   
..         ...           ...                     ...               ...   
95      python    tensorflow  tensorflow-probability          18206662   
96      python    tensorflow                   keras          16824076   
97      python    tensorflow                   keras          11230697   
98      python        pandas                   numpy    